# Download Data

> Module to download the dataset

In [ ]:
# | default_exp preprocess.download_raw_videos

In [ ]:
# | hide
%reload_ext nb_black
%reload_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=

from nbdev.showdoc import *
import sys

__root = "../../"
sys.path.append(__root)

In [ ]:
# | export
from clip_video_classifier.cli import cli
from torch_snippets import *
import subprocess
import yt_dlp

In [ ]:
# | export


def download_youtube_video(url, video_fpath):
    command = ["yt-dlp", "-f", "bestvideo[ext=mp4]", "-o", video_fpath, url]
    subprocess.run(command, check=True)


def try_download_youtube_video(url, video_fpath):
    try:
        download_youtube_video(url, video_fpath)
    except Exception as e:
        Warn(f"Error: {e} @ {video_fpath}")


@cli.command()
def download_raw_videos(annotations_csv_path, folder):
    annotations = pd.read_csv(annotations_csv_path)
    df = annotations[["url", "video"]].drop_duplicates().reset_index(drop=True)
    for ix, row in track2(df.iterrows(), total=len(df)):
        title = row.video
        url = row.url.split("watch?v=")[-1]
        video_fpath = f"{folder}/{title}.mp4"
        if not exists(video_fpath):
            try_download_youtube_video(url, video_fpath)
    else:
        Info(f"Skipping {video_fpath}")

In [ ]:
raw_videos_folder = P("/mnt/347832F37832B388/ml-datasets/ssbd/ssbd-raw-videos")

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
!black {__root}